# Libraries

In [2]:
import pandas as pd
import numpy as np
import astropy
from astropy.io import fits
import os
from PIL import Image
from astropy.time import Time

# Table 

In [10]:
obsrv = pd.read_excel('Цифровой_журнал_Астропластин.xlsx',sheet_name='Кометы')

a = obsrv['Идентификационный номер (номер каробки-номер пластины)'].duplicated()
for i,n in enumerate(a):
    if n  == True:
        print(i)
        print(obsrv['Идентификационный номер (номер каробки-номер пластины)'].iloc[i])
        print('~~~')

for i,ra in enumerate(obsrv['RA']):
    """
    In the RA column there is 'm' in some rows, but there isn't 's'. 
    Therefore let's remove 'm' from the data, cause it will prevent calculations. 
    """
    if 'm' in str(ra):
        obsrv['RA'][i] = obsrv['RA'][i].replace('m','')

    """
    Now we should convert coords from '\~h\~.\~~' to '~~:~~:\~~.\~~' format. 
    We should write ':' because iraf doesn't read coordinates with space.
    """
for i,ra in enumerate(obsrv['RA']):
    if 'h' in str(ra):
        
        # there may be some excess space or ',' instead of '.'
        #also we should remove 'h' by splitting and only then convert values
        ra = ra.replace(' ','').replace(',','.').split('h')
        h = ra[0]
        m = int(float(ra[1]))
        s = round((float(ra[1]) - m) * 60,3) 
        if len(h)<2:
            if m<10:
                if s<10:
                    obsrv['RA'][i] = f'0{h}:0{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'0{h}:0{m}:{s}'
                    
            else:
                if s<10:
                    obsrv['RA'][i] = f'0{h}:{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'0{h}:{m}:{s}'
                
        else:
            if m<10:
                if s<10:
                    obsrv['RA'][i] = f'{h}:0{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'{h} 0{m} {s}'
                    
            else:
                if s<10:
                    obsrv['RA'][i] = f'{h}:{m}:0{s}'
                else:
                    obsrv['RA'][i] = f'{h}:{m}:{s}'
            
    else:
        obsrv['RA'][i] = obsrv['RA'][i]
        
"""
Now let's create new table of objects with only known RA
As we can see, number of unique objects with known RA is 
less than number of all unique objects. 
So we should use list of unique objs with know RA.
Now we will create table with unique objects with known RA 
for repeated objects with unsigned RA.
"""

ra_notnull_table = obsrv[obsrv['RA'].notnull()].reset_index()
#there may be coords without object names, so let's delete such rows
ra_notnull_table = ra_notnull_table[ra_notnull_table['OBJECT'].notnull()].reset_index()

for id_unk,obj_unk in enumerate(obsrv['OBJECT']):
    for id_kn,obj_kn in enumerate(ra_notnull_table['OBJECT']):
        if obj_unk == obj_unk: #if name not Nan
            if obj_unk == obj_kn:
                if obsrv['RA'][id_unk] != ra_notnull_table['RA'][id_kn]: 
                    if obsrv['RA'][id_unk] != obsrv['RA'][id_unk]: #if Nan
                        obsrv['RA'][id_unk] = ra_notnull_table['RA'][id_kn]
                    else: # if not Nan and has own value
                        obsrv['RA'][id_unk] = obsrv['RA'][id_unk]
                else: # if RA are the same
                    continue
        elif obj_unk != obj_unk:#if Nan
            continue
                        
for i,dec in enumerate(obsrv['DEC   (градусы)']):
    if ',' in str(dec):
        obsrv['DEC   (градусы)'][i] = dec.replace(',','.').replace("'","")
        
for i,dec in enumerate(obsrv['DEC   (градусы)']):
    
    if dec == dec:    
        d = int(float(dec))
        m = int((abs(float(dec)) - abs(d)) * 60)
        s = round(((abs(float(dec)) - abs(d)) * 60 - m)*60,3) 

        if d < 0 & d > -10:
            if m<10:
                if s<10:
                    obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:0{m}:0{s}'
                else:
                    obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:0{m}:{s}'

            else:
                if s<10:
                    obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:{m}:0{s}'
                else:
                    obsrv['DEC   (градусы)'][i] = f'-0{abs(d)}:{m}:{s}'

        else:
            if m<10:
                if s<10:
                    obsrv['DEC   (градусы)'][i] = f'{d}:0{m}:0{s}'
                else:
                    obsrv['DEC   (градусы)'][i] = f'{d}:0{m}:{s}'

            else:
                if s<10:
                    obsrv['DEC   (градусы)'][i] = f'{d}:{m}:0{s}'
                else:
                    obsrv['DEC   (градусы)'][i] = f'{d}:{m}:{s}'
                    
dec_notnull_table = obsrv[obsrv['DEC   (градусы)'].notnull()].reset_index()
#there may be coords without object names, so let's delete such rows
dec_notnull_table = dec_notnull_table[dec_notnull_table['OBJECT'].notnull()].reset_index()

for id_unk,obj_unk in enumerate(obsrv['OBJECT']):
    for id_kn,obj_kn in enumerate(dec_notnull_table['OBJECT']):
        if obj_unk == obj_unk: #if not Nan
            if obj_unk == obj_kn:
                if obsrv['DEC   (градусы)'][id_unk] != dec_notnull_table['DEC   (градусы)'][id_kn]: #if DEC values not identic
                    if obsrv['DEC   (градусы)'][id_unk] != obsrv['DEC   (градусы)'][id_unk]: #if Nan
                        obsrv['DEC   (градусы)'][id_unk] = dec_notnull_table['DEC   (градусы)'][id_kn]
                    else: #if not identic and not Nan
                        obsrv['DEC   (градусы)'][id_unk] = obsrv['DEC   (градусы)'][id_unk]
                else:#if DEC values identic
                    continue
        elif obj_unk != obj_unk:
            continue
            
"""
EXPOSITION

We should write expositions in seconds only! So let's convert it
if it is necessary.
"""
obsrv['exp_name'] = obsrv['exposition ']

for e,exp in enumerate(obsrv['exposition ']): #если калиброква без экп заменяем на 'calibr'
#     if 'Calibrovka' in obsrv['OBJECT'][e]:
#         if obsrv['exposition '][e] == obsrv['exposition '][e]:
#             obsrv['exposition '][e] = obsrv['exposition '][e]
#         else:
#             obsrv['exposition '][e]='calibr'
            
    try:
        
        if obsrv['exposition '][e] == obsrv['exposition '][e]:
            obsrv['exposition '][e] = obsrv['exposition '][e].replace(",",";")
            if '*' not in str(exp):
                if ';' in str(exp):
                    obsrv['exposition '][e] = str(exp).split(';')
                    for g in range(0,len(obsrv['exposition '][e])):
                        if 's' in str(obsrv['exposition '][e][g]) and 'm' not in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('s','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g]) * 1
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('m','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g])*60
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' not in str(obsrv['exposition '][e][g]) and 'h' in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('h','')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g])*3600
                        elif 's' in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' not in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('s','').split('m')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g][0])*60 +float(obsrv['exposition '][e][g][1])
                        elif 's' not in str(obsrv['exposition '][e][g]) and 'm' in str(obsrv['exposition '][e][g]) and 'h' in str(obsrv['exposition '][e][g]):
                            obsrv['exposition '][e][g] = str(obsrv['exposition '][e][g]).replace('m','').split('h')
                            obsrv['exposition '][e][g] = float(obsrv['exposition '][e][g][0])*3600 +float(obsrv['exposition '][e][g][1])*60
                else:
                    if 's' in str(exp) and 'm' not in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('s','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e]) * 1
                    elif 'm' in str(exp) and 's' not in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('m','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e])*60
                    elif 's' in str(exp) and 'm' in str(exp) and 'h' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('s','').split('m')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e][0])*60 + float(obsrv['exposition '][e][1])
                    elif 'h' in str(exp) and 'm' not in str(exp) and 's' not in str(exp):
                        obsrv['exposition '][e] = exp.replace('h','')
                        obsrv['exposition '][e] = float(obsrv['exposition '][e])*3600
                    elif 'h' in str(exp) and 'm' in str(exp) and 's' not in str(exp):
                        exp = exp.replace('m','').split('h')
                        obsrv['exposition '][e] = float(exp[0])*3600 + float(exp[1])*60
    except ValueError:
        print(e)

"""
DATE-OBS
"""

lambd = (76+57/60+58/3600)*24/360
obsrv['DATE-OBS-ONE'] = ['']*len(obsrv) #create new column in the table. Here will be writen all dates and time of observation in fits format 
obsrv['DATE-OBS-ONE_2'] = ['']*len(obsrv) #create new column in the table. this one is necessary for cases when we have 2 nights (4 dates)
obsrv['DATE-OBS-JD'] = ['']*len(obsrv) #create new column in the table. in the header we should write date of observations in JD
obsrv['DATE-OBS-JD_2'] = ['']*len(obsrv) #create new column in the table. in the header we should write date of observations in JD/ this one is necessary for cases when we have 2 nights (4 dates)
# try:
    
for i in range(0,len(obsrv)):
    if '*' not in str(obsrv['exposition '][i]) and '*' not in str(obsrv['TIME-OBS (tзв-s)'][i]):#we do not work with polarization, so we exclude observations with * in time-obs
        if obsrv['TIME-OBS (tзв-s)'][i] == obsrv['TIME-OBS (tзв-s)'][i]: #check if value exist
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace('/','.')
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace('?','')
            obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].replace(" ","")

            """
            if we have 2 nights (4 dates)
            """

            if ';' in obsrv['DATE-OBS'][i]: 

                obsrv['TIME-OBS (tзв-s)'][i] = str(obsrv['TIME-OBS (tзв-s)'][i]).replace(',',';') #if we have comma we should replace it with ;, cause we will split by this sybol our 2 nights
                obsrv['TIME-OBS (tзв-s)'][i] = obsrv['TIME-OBS (tзв-s)'][i].split(';')
                time_0 = obsrv['TIME-OBS (tзв-s)'][i][0] #the first night (the first 2 dates)
                time_1 = obsrv['TIME-OBS (tзв-s)'][i][1] # the second night (the last 2 dates)

                #Then we have some comditions for time-obs which includ s or m ot both or any of them. 
                #We should write time in hours without m and s
                if 's' in time_0:
                    time_0 = time_0.replace('s','')
                    time_0 = time_0.split('h')
                    time_0_m = time_0[1].split('m')
                    S1_0 = float(time_0[0]) + float(time_0_m[0])/60 + float(time_0_m[1])/3600 #местное звездное время (данные из таблицы)

                elif 's' not in time_0:

                    if 'm' in time_0:
                        time_0 = time_0.replace('m','').split('h')
                        S1_0 = float(time_0[0]) + float(time_0[1].split('m')[0])/60 #местное звездное время (данные из таблицы)


                    elif 'm' not in str(time_0):
                        time_0 = time_0.split('h')
                        S1_0 = float(time_0[0]) + float(time_0[1])/60 #местное звездное время (данные из таблицы)

                if 's' in time_1:
                    time_1 = time_1.replace('s','')
                    time_1 = time_1.split('h')
                    time_1_m = time_1[1].split('m')
                    S1_1 = float(time_1[0]) + float(time_1_m[0])/60 + float(time_1_m[1])/3600 #местное звездное время (данные из таблицы)

                else:

                    if 'm' in time_1:
                        time_1 = time_1.replace('m','').split('h')
                        S1_1 = float(time_1[0]) + float(time_1[1].split('m')[0])/60 #местное звездное время (данные из таблицы)
                    elif 'm' not in str(time_1):
                        time_1 = time_1.split('h')
                        S1_1 = float(time_1[0]) + float(time_1[1])/60 #местное звездное время (данные из таблицы)

                obsrv['DATE-OBS'][i] = obsrv['DATE-OBS'][i].split(';')
                a_0 = str(obsrv['DATE-OBS'][i][0]).split('.') # '01-02.03.1996' --> '01-02','03','1996' so we have separately days,month and year
                a_0[0] = a_0[0].split('-') # '01-02' --> '01','02' so we have separately two dates

                if len(a_0[2]) < 4: #if we have '96' instead of '1996'
                    a_0[2] = f'19{a_0[2]}'
                D1_0 = f'{a_0[2]}-{a_0[1]}-{a_0[0][0]}' #date1
                D2_0 = f'{a_0[2]}-{a_0[1]}-{a_0[0][1]}' #date2
                JD2_0 = Time(f'{D2_0} 00:00:00').jd     #date2 in JD to future calculations
                T2_0 = (JD2_0 - 2433282.5) / 36524.2    #date in fraction of 100 years
                S0_2_0 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T2_0) + (((0.0929 / 60) / 60) * T2_0**2) #sideral time in Greenwich in UT midnight
                S0_2_0 = S0_2_0 % 24         #to account for only 24 hours in a day
                s_0 = S0_2_0 + lambd         #local sideral time in UT midnight
                s_0 = s_0%24                 #to account for only 24 hours in a day
                sid_h_0 = s_0 - 6            #local sideral time in local midnight
                sid_h_0 = sid_h_0%24         #to account for only 24 hours in a day
                sid_h_2_0 = round(sid_h_0,2) #in the next steps we should find the value in some range, so we should use only 2 values after comma
                S1_0 = S1_0%24               #to account for only 24 hours in a day
                S1_0_2 = round(S1_0,2)       #in the next steps we should find the value in some range, so we should use only 2 values after comma


                grin_mid_0 = Time(f'{D1_0} 18:00:00') #to calculate time in respect with our midnight but 6pm in Greenwich

                a = np.arange(0,12,0.001)
                b = np.arange(12,24,0.001)

                for g,n in enumerate(b):
                    b[g] = round(n,3)

                for g,n in enumerate(a):
                    a[g] = round(n,3)


                if sid_h_2_0 in a: # a=(0,12)

                    if S1_0_2 in b: # b=(12,24) #before mid
                        delta_0 = sid_h_0 + 24 - S1_0
                        if delta_0 > 6:
                            delta_0 = 24 - delta_0
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')      
                            delta_0_time = T1_delt_0 - T2_mid_0  
                            date_0 = grin_mid_0 - delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd
                        else:
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')      
                            delta_0_time = T1_delt_0 - T2_mid_0  
                            date_0 = grin_mid_0 - delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                    elif S1_0_2 in a:

                        if S1_0_2 > sid_h_2_0:#after mid
                            delta_0 =  S1_0 - sid_h_0
                            delta_0 = delta_0%24
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')      
                            delta_0_time = T1_delt_0 - T2_mid_0
                            date_0 = grin_mid_0 + delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                        elif S1_0_2 < sid_h_2_0: #before mid
                            delta_0 =  sid_h_0 - S1_0
                            delta_0 = delta_0%24
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')      
                            delta_0_time = T1_delt_0 - T2_mid_0    
                            date_0 = grin_mid_0 - delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                elif sid_h_2_0 in b: #(12,24)

                    if S1_0_2 in b:

                        if S1_0_2 > sid_h_2_0: #after mid
                            delta_0 =  S1_0 - sid_h_0 
                            delta_0 = delta_0%24
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')      
                            delta_0_time = T1_delt_0 - T2_mid_0
                            date_0 = grin_mid_0 + delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                        elif S1_0_2 < sid_h_2_0: #before mid
                            delta_0 = sid_h_0 - S1_0  
                            delta_0 = delta_0%24
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')     
                            delta_0_time = T1_delt_0 - T2_mid_0  
                            date_0 = grin_mid_0 - delta_0_time
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                    elif S1_0_2 in a: #after mid
                        delta_0 = S1_0 + 24 - sid_h_0
                        if delta_0 > 6:
                            delta_0 = 24 - delta_0
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')           
                            delta_0_time = T1_delt_0 - T2_mid_0    
                            date_0 = grin_mid_0 + delta_time_0
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd
                        else:
                            delta_0_hh = int(delta_0)
                            delta_0_m_float = (delta_0 - delta_0_hh)*60
                            delta_0_mm = int(delta_0_m_float)
                            delta_0_ss = (delta_0_m_float - delta_0_mm)*60 
                            delta_0_ss = int(delta_0_ss)
                            delta_0_format = f'{delta_0_hh}:{delta_0_mm}:{delta_0_ss}'
                            T1_delt_0 = Time(f'{D1_0} {delta_0_format}')
                            T2_mid_0 = Time(f'{D1_0} 00:00:00')           
                            delta_0_time = T1_delt_0 - T2_mid_0    
                            date_0 = grin_mid_0 + delta_time_0
                            obsrv['DATE-OBS-ONE'][i] = date_0.fits
                            obsrv['DATE-OBS-JD'][i] = date_0.jd

                a_1 = str(obsrv['DATE-OBS'][i][1]).split('.') # '01-02.03.1996' --> '01-02','03','1996' so we have separately days,month and year
                a_1[0] = a_1[0].split('-') # '01-02' --> '01','02' so we have separately two dates

                if len(a_1[2]) < 4:#if we have '96' instead of '1996'
                    a_1[2] = f'19{a_1[2]}'

                D1_1 = f'{a_1[2]}-{a_1[1]}-{a_1[0][0]}' #date1
                D2_1 = f'{a_1[2]}-{a_1[1]}-{a_1[0][1]}' #date2
                JD2_1 = Time(f'{D2_1} 00:00:00').jd    #date2 in JD to future calculation
                T2_1 = (JD2_1 - 2433282.5) / 36524.2   #date in fraction of 100 years 
                S0_2_1 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T2_1) + (((0.0929 / 60) / 60) * T2_1**2) #sideral time in Greenwich in UT midnight
                S0_2_1 = S0_2_1 % 24   #to account for only 24 hours in a day
                s_1 = S0_2_1 + lambd  #local sideral time in UT midnight
                s_1 = s_1%24  #to account for only 24 hours in a day
                sid_h_1 = s_1 - 6  #local sideral time in local midnight
                sid_h_1 = sid_h_1%24  #to account for only 24 hours in a day
                sid_h_2_1 = round(sid_h_1,2)         #in the next steps we should find the value in some range, so we should use only 2 values after comma
                S1_1 = S1_1%24  #to account for only 24 hours in a day
                S1_1_2 = round(S1_1,2)  #in the next steps we should find the value in some range, so we should use only 2 values after comma

                grin_mid_1 = Time(f'{D1_1} 18:00:00')

                a = np.arange(0,12,0.001)
                b = np.arange(12,24,0.001)

                for g,n in enumerate(b):
                    b[g] = round(n,3)

                for g,n in enumerate(a):
                    a[g] = round(n,3)

                if sid_h_2_1 in a: # a=(0,12)

                    if S1_1_2 in b: # b=(12,24) #before mid
                        delta_1 = sid_h_1 + 24 - S1_1
                        if delta_1 > 6:
                            delta_1 = 24 - delta_1
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')      
                            delta_1_time = T1_delt_1 - T2_mid_1  
                            date_1 = grin_mid_1 - delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd
                        else:
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')      
                            delta_1_time = T1_delt_1 - T2_mid_1  
                            date_1 = grin_mid_1 - delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd


                    elif S1_1_2 in a:

                        if S1_1_2 > sid_h_2_1:#after mid
                            delta_1 =  S1_1 - sid_h_1
                            delta_1 = delta_1%24
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')      
                            delta_1_time = T1_delt_1 - T2_mid_1
                            date_1 = grin_mid_1 + delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd

                        elif S1_1_2 < sid_h_2_1: #before mid
                            delta_1 =  sid_h_1 - S1_1
                            delta_1 = delta_1%24
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')      
                            delta_1_time = T1_delt_1 - T2_mid_1    
                            date_1 = grin_mid_1 - delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd

                elif sid_h_2_1 in b: #(12,24)

                    if S1_1_2 in b:

                        if S1_1_2 > sid_h_2_1: #after mid
                            delta_1 =  S1_1 - sid_h_1 
                            delta_1 = delta_1%24
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')      
                            delta_1_time = T1_delt_1 - T2_mid_1
                            date_1 = grin_mid_1 + delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd

                        elif S1_1_2 < sid_h_2_1: #before mid
                            delta_1 = sid_h_1 - S1_1  
                            delta_1 = delta_1%24
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')     
                            delta_1_time = T1_delt_1 - T2_mid_1  
                            date_1 = grin_mid_1 - delta_1_time
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd

                    elif S1_1_2 in a: #after mid
                        delta_1 = S1_1 + 24 - sid_h_1
                        if delta_1 > 6:
                            delta = 24 - delta
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')           
                            delta_1_time = T1_delt_1 - T2_mid_1    
                            date_1 = grin_mid_1 + delta_time_1
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits 
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd
                        else:
                            delta_1_hh = int(delta_1)
                            delta_1_m_float = (delta_1 - delta_1_hh)*60
                            delta_1_mm = int(delta_1_m_float)
                            delta_1_ss = (delta_1_m_float - delta_1_mm)*60 
                            delta_1_ss = int(delta_1_ss)
                            delta_1_format = f'{delta_1_hh}:{delta_1_mm}:{delta_1_ss}'
                            T1_delt_1 = Time(f'{D1_1} {delta_1_format}')
                            T2_mid_1 = Time(f'{D1_1} 00:00:00')           
                            delta_1_time = T1_delt_1 - T2_mid_1    
                            date_1 = grin_mid_1 + delta_time_1
                            obsrv['DATE-OBS-ONE_2'][i] = date_1.fits
                            obsrv['DATE-OBS-JD_2'][i] = date_1.jd
                            
#             """
#             if we have 1 night (2 dates)
#             """

            elif ';' not in obsrv['DATE-OBS'][i]: 

                if obsrv['TIME-OBS (tзв-s)'][i] == obsrv['TIME-OBS (tзв-s)'][i]: #проверяем на существование значения
                    obsrv['TIME-OBS (tзв-s)'][i] = str(obsrv['TIME-OBS (tзв-s)'][i]).replace(',',';')
                    obsrv['TIME-OBS (tзв-s)'][i] = obsrv['TIME-OBS (tзв-s)'][i].split(';')
                    time = obsrv['TIME-OBS (tзв-s)'][i][0]

                    if 's' in time:
                        time = time.replace('s','')
                        time = time.split('h')
                        time_m = time[1].split('m')
                        S1 = float(time[0]) + float(time_m[0])/60 + float(time_m[1])/3600 #местное звездное время (данные из таблицы)
                        time_sid_table_t = f'{time[0]}:{time_m[0]}:{time_m[1]}'
                        time_sid_table_t = time_sid_table_t.replace(" ","")


                    elif 's' not in time:
                        if 'm' in time:
                            time = time.replace('m','').split('h')
                            S1 = float(time[0]) + float(time[1].split('m')[0])/60 #местное звездное время (данные из таблицы)

                        elif 'm' not in str(time):
                            time = time.split('h')
                            S1 = float(time[0]) + float(time[1])/60 #местное звездное время (данные из таблицы)

                l = str(obsrv['DATE-OBS'][i]).split('.')
                
                """
                if there are two month, for example 31.03-01.04.1996
                """
                
                if len(l) > 3:
                    a = str(obsrv['DATE-OBS'][i]).split('-')
                    a[0] = a[0].split('.')
                    a[1] = a[1].split('.')
                    
                    try:
                    
                        if len(a[1][2]) < 4:#if we have '96' instead of '1996'
                            a[1][2] = f'19{a[1][2]}'
                        D1 = f'{a[1][2]}-{a[0][1]}-{a[0][0]}' #date1 
                        D2 = f'{a[1][2]}-{a[1][1]}-{a[1][0]}'  #date2
                        JD2 = Time(f'{D2} 00:00:00').jd #date2 in JD to future calculation
                        T2 = (JD2 - 2433282.5) / 36524.2#date in fraction of 100 years 
                        S0_2 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T2) + (((0.0929 / 60) / 60) * T2**2) #sideral time in Greenwich in UT midnight
                        S0_2 = S0_2 % 24  #to account for only 24 hours in a day
                        s_ = S0_2 + lambd  #local sideral time in UT midnight
                        s_ = s_%24  #to account for only 24 hours in a day
                        sid_h = s_ - 6  #local sideral time in local midnight
                        sid_h0 = sid_h%24  #to account for only 24 hours in a day
                        sid_h_2 = round(sid_h0,2)  #in the next steps we should find the value in some range, so we should use only 2 values after comma
                        S1 = S1%24  #to account for only 24 hours in a day
                        S1_2 = round(S1,2)  #in the next steps we should find the value in some range, so we should use only 2 values after comma

                        grin_mid = Time(f'{D1} 18:00:00')

                        a = np.arange(0,12,0.001)
                        b = np.arange(12,24,0.001)

                        for g,n in enumerate(b):
                            b[g] = round(n,3)
                        for g,n in enumerate(a):
                            a[g] = round(n,3)

                        if sid_h_2 in a: # a=(0,12)

                            if S1_2 in b: # b=(12,24) #before mid
                                delta = sid_h0 + 24 - S1
                                if delta > 6:
                                    delta = 24 - delta
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid  
                                    date = grin_mid - delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd
                                else: 
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid  
                                    date = grin_mid - delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd

                            elif S1_2 in a:

                                if S1_2 > sid_h_2:#after mid
                                    delta =  S1 - sid_h0
                                    delta = delta%24
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid   
                                    date = grin_mid + delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits

                                elif S1_2 < sid_h_2: #before mid
                                    delta =  sid_h0 - S1
                                    delta = delta%24
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid        
                                    date = grin_mid - delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd

                        elif sid_h_2 in b: #(12,24)

                            if S1_2 in b:

                                if S1_2 > sid_h_2: #after middelta =  S1 - sid_h0 
                                    delta = delta%24
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid        
                                    date = grin_mid + delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd

                                elif S1_2 < sid_h_2: #before mid
                                    delta = sid_h0 - S1  
                                    delta = delta%24
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid       
                                    date = grin_mid - delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd

                            elif S1_2 in a: #after mid
                                delta = S1 + 24 - sid_h0
                                if delta > 6:
                                    delta = 24 - delta
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid        
                                    date = grin_mid + delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd
                                else:
                                    delta_hh = int(delta)
                                    delta_m_float = (delta - delta_hh)*60
                                    delta_mm = int(delta_m_float)
                                    delta_ss = (delta_m_float - delta_mm)*60 
                                    delta_ss = int(delta_ss)
                                    delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                    T1_delt = Time(f'{D1} {delta_format}')
                                    T2_mid = Time(f'{D1} 00:00:00')      
                                    delta_time = T1_delt - T2_mid        
                                    date = grin_mid + delta_time
                                    obsrv['DATE-OBS-ONE'][i] = date.fits
                                    obsrv['DATE-OBS-JD'][i] = date.jd
                    except IndexError:
                        print(i)

#                 """
#                 if we have 1 month, for example 02-03.06.1993
#                 """
                else:
                    a = str(obsrv['DATE-OBS'][i]).split('.')
                    a[0] = a[0].split('-')
                    if len(a[2]) < 4:#if we have '96' instead of '1996'
                        a[2] = f'19{a[2]}'
                    D1 = f'{a[2]}-{a[1]}-{a[0][0]}' #date1
                    D2 = f'{a[2]}-{a[1]}-{a[0][1]}'#date2
                    JD2 = Time(f'{D2} 00:00:00').jd #date2 in JD to future calculations
                    T2 = (JD2 - 2433282.5) / 36524.2 #date in fraction of 100 years
                    S0_2 = (6 + (40 / 60) + ((18.130 / 60) / 60)) + (8640184.635 /60 / 60 * T2) + (((0.0929 / 60) / 60) * T2**2) #sideral time in Greenwich in UT midnight
                    S0_2 = S0_2 % 24   #to account for only 24 hours in a day
                    s_ = S0_2 + lambd   #local sideral time in UT midnight
                    s_ = s_%24  #to account for only 24 hours in a day
                    sid_h = s_ - 6   #local sideral time in local midnight
                    sid_h0 = sid_h%24  #to account for only 24 hours in a day
                    sid_h_2 = round(sid_h0,2)  #in the next steps we should find the value in some range, so we should use only 2 values after comma
                    S1 = S1%24  #to account for only 24 hours in a day
                    S1_2 = round(S1,2)  #in the next steps we should find the value in some range, so we should use only 2 values after comma

                    grin_mid = Time(f'{D1} 18:00:00')

                    a = np.arange(0,12,0.001)
                    b = np.arange(12,24,0.001)

                    for g,n in enumerate(b):
                        b[g] = round(n,3)

                    for g,n in enumerate(a):
                        a[g] = round(n,3)

                    if sid_h_2 in a: # a=(0,12)

                        if S1_2 in b: # b=(12,24) #before mid
                            delta = sid_h0 + 24 - S1
                            if delta > 6:
                                delta = 24 - delta
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid  
                                date = grin_mid - delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd
                            else:    
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid  
                                date = grin_mid - delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd

                        elif S1_2 in a:

                            if S1_2 > sid_h_2:#after mid
                                delta =  S1 - sid_h0
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid        
                                date = grin_mid + delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd

                            elif S1_2 < sid_h_2: #before mid
                                delta =  sid_h0 - S1
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid        
                                date = grin_mid - delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd

                    elif sid_h_2 in b: #(12,24)

                        if S1_2 in b:

                            if S1_2 > sid_h_2: #after mid
                                delta =  S1 - sid_h0 
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid        
                                date = grin_mid + delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd

                            elif S1_2 < sid_h_2: #before mid
                                delta = sid_h0 - S1  
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid       
                                date = grin_mid - delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd

                        elif S1_2 in a: #after mid
                            delta = S1 + 24 - sid_h0 
                            if delta > 6:
                                delta = 24 - delta
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid
                                date = grin_mid + delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits  
                                obsrv['DATE-OBS-JD'][i] = date.jd
                            else:
                                delta = delta%24
                                delta_hh = int(delta)
                                delta_m_float = (delta - delta_hh)*60
                                delta_mm = int(delta_m_float)
                                delta_ss = (delta_m_float - delta_mm)*60 
                                delta_ss = int(delta_ss)
                                delta_format = f'{delta_hh}:{delta_mm}:{delta_ss}'
                                T1_delt = Time(f'{D1} {delta_format}')
                                T2_mid = Time(f'{D1} 00:00:00')      
                                delta_time = T1_delt - T2_mid
                                date = grin_mid + delta_time
                                obsrv['DATE-OBS-ONE'][i] = date.fits
                                obsrv['DATE-OBS-JD'][i] = date.jd
# except ValueError:
#     print(i)
                                
                
# """
# RUS ---> ENG
# """

#TELESCOPE

for i,tel in enumerate(obsrv['TELESCOPE']):
    if  tel == '50cm менисковый телескоп Максутова':
        obsrv['TELESCOPE'][i] = 'Wide aperture Maksutov meniscus telescope with main mirror 50 cm'
    elif tel != tel:
        obsrv['TELESCOPE'][i] = 'Wide aperture Maksutov meniscus telescope with main mirror 50 cm'

#OBSERVER

observers_uniq = obsrv['OBSERVER'].unique() #to know each unique observer
#then 
observers_dict = {'Рожковский Д.А.': 'Rozhkovskij D.A.', 'Торопова Т.А.':'Tropova T.A.', 
                  'Городецкий Д.И.':'Gordetskij D.I.', 'Глушков Ю.И.':'Glushkovskij Yu.I.', 
                  'Рожковский Д.А., Торопова Т.А.' : 'Rozhkovskij D.A., Tropova T.A.', 'Рожковский Д.А., Павлова Л.А.' : 'Rozhkovskij D.A., Pavlova L.A.',
                  'Карягина З.В.':'Karyagina Z.V.', 'Матягин В.С.': 'Matyagin V.S.', 'Павлова Л.А':'Pavlova L.A.', 'Гаврилов':'Gavrilov', 
                  'Курчаков А.В.':'Kurchakov A.V.'}
obsrv = obsrv.replace({"OBSERVER":observers_dict})

##  HEADER

In [11]:
used_rows = [] #number of rows which were used for files
os.chdir('') #directory where frames are located
counter = 0

for f in os.listdir():
    hdul =  fits.open(f)
    file_name = f.replace('.fits','').replace(".fit","")
    file_name_splitted = file_name.split('_')
    
    for i in range(0,len(obsrv)):
        if file_name_splitted[3] == str(obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]):
            used_rows.append(i)
            if obsrv['Идентификационный номер (номер каробки-номер пластины)'][i] == obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]:
                hdul[0].header['IDN']=obsrv['Идентификационный номер (номер каробки-номер пластины)'][i]  
            
            hdul[0].header['OBJECT']=file_name_splitted[0]
            
            if obsrv['DATE-OBS-ONE'][i] == obsrv['DATE-OBS-ONE'][i]:
                hdul[0].header['DATE-OBS']=obsrv['DATE-OBS-ONE'][i] 
            if obsrv['DATE-OBS-ONE_2'][i] != '':
                hdul[0].header['DATE-OB2']=obsrv['DATE-OBS-ONE_2'][i]    
            
            try:
                if obsrv['exposition '][i] == obsrv['exposition '][i]:
                    n = len(obsrv['exposition '][i])                          
                    for g in range(0,n):
                        hdul[0].header[f'EXPTIME{g+1}']=obsrv['exposition '][i][g]                                    
            except TypeError:
                hdul[0].header[f'EXPTIME']=obsrv['exposition '][i]
                
            if obsrv['RA'][i] == obsrv['RA'][i]:
                hdul[0].header['OBJCTRA']=obsrv['RA'][i]                                                   
            
            if obsrv['DEC   (градусы)'][i] == obsrv['DEC   (градусы)'][i]:
                hdul[0].header['OBJCTDEC']=obsrv['DEC   (градусы)'][i] 
                
            hdul[0].header['EPOCH'] = 1950
                
            if obsrv['DATE-OBS-JD'][i] == obsrv['DATE-OBS-JD'][i]:
                hdul[0].header['JD']=obsrv['DATE-OBS-JD'][i] 
            if obsrv['DATE-OBS-JD_2'][i] != '':
                hdul[0].header['JD2']=obsrv['DATE-OBS-JD_2'][i]  
                
#             hdul[0].header['AIRMASS'] = 1.0 
                
            if obsrv['TELESCOPE'][i] == obsrv['TELESCOPE'][i]:
                hdul[0].header['TELESCOP'] = obsrv['TELESCOPE'][i]          
            
            if obsrv['OBSERVER'][i] == obsrv['OBSERVER'][i]:
                hdul[0].header['OBSERVER'] = obsrv['OBSERVER'][i]
                        
            hdul[0].header['OBSERVAT'] = 'FAI'  
            
            os.remove(f)         
            hdul.writeto(f'{file_name}.fits') #SAVE FILES
            print(hdul[0].header)
            print(f'Info about file {file_name} is located in table row number {i}')
    counter += 1

---
IC5146_04-05.08.1987_50m_3-6141-b.fit.fits

1
~~~~~~~~~~~
---
Sky-Lac-St_04-05.08.1987_50m_3-6141-a.fit.fits

2
~~~~~~~~~~~
---
Calibrovka_29-30.08.1989__3-6436-1-Сal.fit.fits

3
~~~~~~~~~~~
---
M8-NGC6523_21-22.04.1953_14m30s_3-766-a.fit.fits

4
~~~~~~~~~~~
---
Sky-Lac-St_29-30.08.1989_10m_3-6436-b.fit.fits

5
~~~~~~~~~~~
---
Sky-Lac-St_25-26.12.1984_45m_3-5448-a.fit.fits

6
~~~~~~~~~~~
---
Calibrovka_02-03.09.1989__3-6437-Cal.fit.fits

7
~~~~~~~~~~~
---
Calibrovka_23-24.11.1984__3-5444-Cal.fit.fits

8
~~~~~~~~~~~
---
Sky-Lac-St_26-27.08.1987_40m_3-6183-b.fit.fits

9
~~~~~~~~~~~
---
Sky-Lac-St_24-25.11.1984_10m_3-5445-46.fit.fits

10
~~~~~~~~~~~
---
Sky-Nua-St_03-04.09.1989_15m_3-6406-a.fit.fits

11
~~~~~~~~~~~
---
Sky-Lac-St_23-24.10.1986_45m_3-5784-b.fit.fits

12
~~~~~~~~~~~
---
Calibrovka_03-04.03.1989__3-6406-Cal.fit.fits

13
~~~~~~~~~~~
---
Sky-Lac-St_2-3.09.1989_40m_3-6437-b.fit.fits

14
~~~~~~~~~~~
---
M45-Stand_21-22.02.1952_1h_3-415.fit.fits

15
~~~~~~~~~~~
